In [1]:
# This code is used to design sgRNAs based on the PAM sequences.
# It also predicts the associated base editing outcomes.
# please also cite Hwang, G.-H. et al. Web-based design and analysis tools for CRISPR base editing. BMC Bioinformatics 19, 542 (2018).

In [1]:
import os, re
import pandas as pd
import numpy as np
import pysam

In [2]:
def reverse_complement(s):
    return str(s).translate(str.maketrans('ATGC','TACG'))[::-1]

In [3]:
def finditer_everything(pattern, string):
    pattern = re.compile(pattern)
    pos = 0
    m = pattern.search(string, pos)
    while m is not None:
        yield m
        pos = m.start() + 1
        m = pattern.search(string, pos)

In [4]:
def target_yield(query_seq, seed_len, pam_seq):

    pattern = seed_len*'N' + pam_seq
    pattern_rev = pattern.translate(t)[::-1]
    pattern = "(" + pattern + ")|(" + pattern_rev + ")"
    pattern = pattern.replace('N', '[AGTC]').replace('R', '[AG]').replace('W', '[AT]').replace('M', '[AC]').replace('Y', '[CT]').replace('S', '[GC]').replace('K', '[GT]').replace('B', '[CGT]').replace('D', '[AGT]').replace('H', '[ACT]').replace('V', '[ACG]')
    pattern_rev = pattern_rev.replace('N', '[AGTC]').replace('R', '[AG]').replace('W', '[AT]').replace('M', '[AC]').replace('Y', '[CT]').replace('S', '[GC]').replace('K', '[GT]').replace('B', '[CGT]').replace('D', '[AGT]').replace('H', '[ACT]').replace('V', '[ACG]')
    p_rev = re.compile(pattern_rev)

    revmatch = False
    for m in finditer_everything(pattern, query_seq):
        for i in (1, 2): 
            # combined (0), forward (1) and reverse (2) matching
            # When there is a match, or a match on both ends (revisit).
            if m.group(i) or revmatch:
                if i == 1:
                    seq_match = m.group(i)
                    direction = '+'
                    # If the sgRNA can be also matched in the rev direction,
                    # it will show up in group 1 only but we will need to revist.
                    if p_rev.match(m.group(i)) is not None:
                        revmatch = True # This labels future revisit
                else:
                    if revmatch:
                        seq_match = m.group(1).translate(t)[::-1]
                        revmatch = False
                    else:
                        seq_match = m.group(i).translate(t)[::-1]
                    direction = '-'
                # target seq in + strand, guide direction, actual target seq, start pos (0-index)
                yield (m.group(), direction, seq_match, m.start())

In [5]:
def find_targets(query_seq, ref_base, alt_base):
    targets = []
    for target in target_yield(query_seq, seed_len, pam_seq):
        pos = int(target[3])
        #both start and end are included in the x-nt window
        if target[1] == '+': # guide strand
            window_start_abs = pos + seed_len - window_ed
            window_end_abs = pos + seed_len - window_st          

        else:
            window_start_abs = pos + pam_len -1 + window_st
            window_end_abs = pos + pam_len - 1 + window_ed
            
        # Adjust the extra adaptor intron sequences.
        window_start_abs -= adapt_len
        window_end_abs -= adapt_len
        
        # There is adaptor intron sequence in the input. 
        # Window start/end are based on the middle exon seq
        # Anything outside the exon window needs to be removed.
        if window_end_abs <= -1 or window_start_abs >= (len(query_seq) - 2*adapt_len):
            window_seq = ''
        else:
            window_start_abs = max(window_start_abs, 0)
            # It is "-2*adapt_len -1"
            window_end_abs = min(window_end_abs, len(query_seq) - 2*adapt_len - 1)
            # Note the window seq always refers to the + strand.
            # The DNA at window_end_abs will be included.
            window_seq = query_seq[adapt_len:][window_start_abs: window_end_abs + 1]
        
        mutated_window_seq = ''
        for i in range(len(window_seq)):
            if window_seq[i] == ref_base and target[1] == '+':
                mutated_window_seq += alt_base
            elif window_seq[i] == reverse_complement(ref_base) and target[1] == '-':
                mutated_window_seq += reverse_complement(alt_base)
            else:
                mutated_window_seq += window_seq[i]

        window_pos = [window_start_abs, window_end_abs]
        
        if mutated_window_seq != window_seq:
            if window_seq != '':
                #target seq in + strand:[0], guide strand / direction:[1], 
                # actual target seq:[2], start pos (0-index):[3]
                # Note: both window_seq and mutated_window_seq
                # are based on + ref strand seq.
                targets.append([target[2], 
                                target[3]-adapt_len, 
                                target[1], 
                                window_seq, 
                                mutated_window_seq, 
                                window_pos])
        
    return(targets)

In [6]:
def compare_aa(original_CDS, mutated_CDS):
    if len(original_CDS) != len(mutated_CDS) or len(original_CDS)%3 !=0:
        print('The input CDS is wrong.')
    aa_len = int(len(original_CDS) // 3)
    aa_change = []
    for i in range(aa_len):
        from_aa = amino_sym[original_CDS[3*i:(3*i+3)]]
        to_aa = amino_sym[mutated_CDS[3*i:(3*i+3)]]
        if from_aa != to_aa:
            # The position is 1-indexed in protein
            aa_change.append([(i+1), from_aa, to_aa])
    return aa_change

In [7]:
def extract_genome_seq(chromosome, start_pos, end_pos):
    query_seq = ref_genome.fetch(chromosome, start_pos-1, end_pos)
    return query_seq

In [8]:
def design_guides(data, ref_base, alt_base):
    exon_count = data.shape[0]
    exon_seqs_original = []
    # This will collect data based on different exons.
    combined_targets = []

    for ex in range(exon_count):
        exon_seqs_original += [extract_genome_seq(chromosome='chr'+str(data.iloc[ex, 2]),
                                       start_pos=data.iloc[ex, 3], 
                                       end_pos=data.iloc[ex, 4])]
        
    for ex in range(exon_count):
        query_seq = extract_genome_seq(chromosome='chr'+str(data.iloc[ex, 2]),
                                       start_pos=data.iloc[ex, 3]-21, 
                                       end_pos=data.iloc[ex, 4]+21)
        strands = set(data.iloc[:, 5].values.tolist())
        if len(strands) != 1:
            print('Strand definition is wrong')
        else:
            strand = list(strands)[0]

        ## Run function to get the target sgRNAs.
        targets = find_targets(query_seq, ref_base, alt_base)
        
        for sg in range(len(targets)):
            exon_seqs = exon_seqs_original.copy()
            window_start_abs = targets[sg][5][0]
            window_end_abs = targets[sg][5][1]
            # This is the gene strand    
            if strand == '+':
                original_CDS = ''.join(exon_seqs)
                mutated_exon = exon_seqs[ex][:window_start_abs] + targets[sg][4] + exon_seqs[ex][(window_end_abs+1):]
                exon_seqs[ex] = mutated_exon
                mutated_CDS = ''.join(exon_seqs)
            
            # If the protein is encoded by the - strand.
            # The reverse complement of CDS is needed.
            else:
                original_CDS = ''.join([reverse_complement(kk) for kk in exon_seqs])
                mutated_exon = exon_seqs[ex][:window_start_abs] + targets[sg][4] + exon_seqs[ex][(window_end_abs+1):]
                exon_seqs[ex] = mutated_exon
                mutated_CDS = ''.join([reverse_complement(kk) for kk in exon_seqs])
            
            aa_result = compare_aa(original_CDS, mutated_CDS)        
            combined_targets.append([data.iloc[ex, 1]] +
                                    [data.iloc[ex, 2]] +
                                    [data.iloc[ex, 3]] +
                                    [data.iloc[ex, 4]] +
                                    [data.iloc[ex, 5]] +
                                    [data.iloc[ex, 0]] +
                                    targets[sg] + 
                                    [aa_result])
    return combined_targets

In [9]:
def create_library(ENSP_db_path, output_folder, output_name, ref_base, alt_base, ID):

    input_data = pd.read_csv(ENSP_db_path, dtype={"chr": "string"})
    input_data = input_data.rename(columns={'Unnamed: 0': 'exon_num'})
    #input_data.head()

    unique_ENSP = input_data.loc[:, 'ENSP'].unique()
    input_data.index = input_data.loc[:, 'ENSP']
    col_names = ['ENSP', 'chr', 'genome_start', 'genome_end', 'gene_strand',
                'exon_num','target_seq_wPAM', 'st_pos_in_exon', 
                'guide_strand', 'window_seq_pos_strand','mutated_window_seq_pos_strand', 
                'window_pos_index0', 'aa_pos_from_to']

    # Create an empty df
    combined_results_df = pd.DataFrame(columns = col_names)
    #len(unique_ENSP)
    for i in range(len(unique_ENSP)):
        if (i%100==0):
            print(i)
        ENSP_data = input_data.loc[[unique_ENSP[i]], :]
        results = design_guides(ENSP_data, ref_base, alt_base)
        results_df = pd.DataFrame(results)
        results_df.columns = col_names
        combined_results_df = combined_results_df.append(results_df)

    # Reindex the df
    combined_results_df.index = list(range(combined_results_df.shape[0]))
    
    # save the results
    combined_results_df.to_pickle(os.path.join(output_folder, output_name+str(ID)+".pkl"))
    #combined_results_df.to_csv(os.path.join(output_folder, output_name+".csv"))


In [11]:
# Set the parameters that will be used globally.
seed_len = 20
pam_seq = 'NGNN'
amino_sym = {"TGG": "W", "GGG": "G", "GTT": "V", "CGG": "R", "ACT": "T", "TGA": "X", "CTA": "L", "TCC": "S","GAA": "E", "CCA": "P", "GAC": "D", "ACC": "T", "TTT": "F", "CTC": "L", "GCT": "A", "CCC": "P", "TCG": "S", "CAT": "H", "GTC": "V", "CGA": "R", "CAG": "Q", "ATA": "I", "AAG": "K", "CCG": "P", "GGA": "G", "AGC": "S", "TAT": "Y", "CTG": "L", "ACG": "T", "GAG": "E", "GCT": "A", "TGC": "C", "TGT": "C", "AGG": "R", "ATG": "M", "TTA": "L", "GCA": "A", "AAT": "N", "GTA": "V", "GGT": "G", "AGA": "R", "CGC": "R", "ATC": "I", "TAC": "Y", "TTG": "L", "ACA": "T", "GCG": "A", "CTT": "L", "ATT": "I", "CGT": "R", "CAC": "H", "TCA": "S", "CCT": "P", "TAA": "X", "GAT": "D", "GTG": "V", "AAA": "K", "AAC": "N", "GGC": "G", "TTC": "F", "CAA": "Q", "AGT": "S", "TAG": "X", "TCT": "S", "GCC": "A"}
ref_genome = pysam.FastaFile("data/GRCh38.p7.genome.fa")
t = str.maketrans("ATGCRYSWKMBDHV", "TACGYRWSMKVHDB")
pam_len = len(pam_seq)
adapt_len = 21
window_st = 13 # 10 for evoCDA, 13 for ABE8e/BE4max.
window_ed = 17 # 20 for evoCDA, 17 for ABE8e/BE4max.

ENSP_db_path = "data/ENSP_input_example.txt"
output_folder = 'data/'

create_library(ENSP_db_path, output_folder, output_name='all_CBE_', ref_base='C', alt_base='T', ID=1)
create_library(ENSP_db_path, output_folder, output_name='all_ABE_', ref_base='A', alt_base='G', ID=1)


0
0
